# Домашнее задание № 5 

В данном задании требуется реализовать некоторые из метрик, рассмотренные на лекции.

Все функции, кроме ```compute_gain```, в качестве первых двух аргументов принимают на вход тензоры ```ys_true``` и ```ys_pred```. Это вещественные тензоры ```pytorch``` размерности ```n```, задающие целевые отметки релевантности и предсказанные значения соответственно. 

Для генерации примеров входных данных можете использовать ```torch.rand(n)```, если не указана специфика исходных тензоров. 

Считается, что ```ys_pred``` содержит уникальные значения без повторений.

In [2]:
import math
from math import log2

import torch
from torch import Tensor, sort

## Swapped Pairs

```num_swapped_pairs``` — функция для расчёта количества неправильно упорядоченных пар (корректное упорядочивание — от наибольшего значения в ```ys_true``` к наименьшему) или переставленных пар.

In [2]:
def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    idxs = torch.argsort(ys_pred, descending=True)
    true_sorted = ys_true[idxs]
    
    count = 0
    for i in range(len(ys_true)):
        count += torch.sum(true_sorted[i:] > true_sorted[i])
    
    return int(count)

In [3]:
# не изменять
ys_true = torch.tensor([2, 1, 0, 1, 2])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.13, 0.12])

res = num_swapped_pairs(ys_true, ys_pred)
print(res)  # 7

7


## Gain

```compute_gain``` — вспомогательная функция для расчёта DCG и NDCG, рассчитывающая показатель Gain. Принимает на вход дополнительный аргумент — указание схемы начисления Gain (```gain_scheme```).

Необходимо реализовать метод при:
- ```gain_scheme="const"``` - постоянное начисление Gain
- ```gain_scheme="exp2"``` - рассчитываемый по формуле $(2^r −1)$, где $r$ — реальная релевантность документа некоторому запросу.

In [5]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == 'const':
        return float(y_value)

    elif gain_scheme == 'exp2':
        return float(2 ** y_value - 1)
    
    return float("inf")

In [6]:
# не изменять
value = 5

res = compute_gain(value, 'exp2')
print(res)  # 31

res = compute_gain(value, 'const')
print(res)  # 5

31.0
5.0


## DCG

```dcg``` и ```ndcg``` — функции расчёта DCG и NDCG. Принимают на вход дополнительный параметр ```gain_scheme```, аналогичный таковому в функции ```compute_gain```.

In [21]:
def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str, top_k: int = 0) -> float:
    idx = torch.argsort(ys_pred, descending=True)
    true_sorted = ys_true[idx].to(torch.float64)[:top_k]
    
    steps = torch.arange(2, top_k + 2, dtype=torch.float64)
    steps = torch.log2(steps)

    gains = true_sorted.apply_(lambda x: compute_gain(x, gain_scheme))
    return float(torch.sum(gains / steps))
    

def ndcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str = 'const', top_k: int = 0) -> float:
    dcg_score = dcg(ys_true, ys_pred, gain_scheme, top_k)
    ideal_dcg = dcg(ys_true, ys_true, gain_scheme, top_k)

    return float(dcg_score / ideal_dcg)

In [22]:
pred = torch.tensor([1, 1, 3, 2, 2, 3, 2, 1, 2, 2, 2])
true, _ = torch.tensor(pred).sort(descending=True)

ndcg(true, pred, 'const', 5)

tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
tensor([1.0000, 1.5850, 2.0000, 2.3219, 2.5850], dtype=torch.float64)
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
tensor([3., 3., 2., 2., 2.], dtype=torch.float64)
tensor([1.0000, 1.5850, 2.0000, 2.3219, 2.5850], dtype=torch.float64)
tensor([3., 3., 2., 2., 2.], dtype=torch.float64)


c:\Max\Proga\GRISHIN_ML\ml_mipt_dafe\homeworks\hw_6\.venv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


0.7833471457646615

In [7]:
# не изменять
ys_true = torch.tensor([2, 2, 4, 1, 2, 0])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = dcg(ys_true, ys_pred, gain_scheme='exp2')
res_n = ndcg(ys_true, ys_pred, gain_scheme='exp2')
print(res)  # 12.052645801815459
print(res_n)  # 0.6004804162123548

12.052645801815459
0.6004804162123549


## Precission@k


```precission_at_k``` — функция расчёта точности в топ-k позиций для бинарной разметки (в ```ys_true``` содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то необходимо вернуть -1. 

Функция принимает на вход параметр k, указывающий на то, по какому количеству объектов необходимо произвести расчёт метрики. Учтите, что k может быть больше количества элементов во входных тензорах.

In [8]:
def precission_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    if sum(ys_true)==0:
        return -1
    k = min(len(ys_pred), k)
    sort_idxs = torch.argsort(ys_pred, descending=True)
    pred_sorted = ys_true[sort_idxs]
    true_positive = torch.sum(pred_sorted[:k] == 1)

    return float(true_positive / k) if true_positive > 0 else -1

In [9]:
def recall_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    if (k<=0):
        return float(0)
    
    k = min(len(ys_pred), k)
    sort_idxs = torch.argsort(ys_pred, descending=True)
    pred_sorted = ys_true[sort_idxs]
    true_positive = torch.sum(pred_sorted[:k] == 1)

    return float(true_positive / torch.sum(ys_true))

In [10]:
# не изменять
ys_true = torch.tensor([0, 0, 1, 1, 0, 1])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = precission_at_k(ys_true, ys_pred, k=3)
print(res)  # 0.6666666865348816

0.6666666865348816


## Average Precision

```average_precision``` — функция расчёта AP для бинарной разметки (в ```ys_true``` содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то необходимо вернуть -1.

In [11]:
def average_precision(ys_true: Tensor, ys_pred: Tensor) -> float:
    if sum(ys_true)==0:
        return -1
    AP = 0
    for k in range(1, len(ys_true)+1):
        print("------------------------")
        print(f"Recall {k=}:     ", recall_at_k(ys_true, ys_pred, k))
        print(f"Recall {k-1=}:   ", recall_at_k(ys_true, ys_pred, k-1))
        print(f"precission {k=}: ", precission_at_k(ys_true, ys_pred, k))
        AP += (
            recall_at_k(ys_true, ys_pred, k) - recall_at_k(ys_true, ys_pred, k-1)
        ) * precission_at_k(ys_true, ys_pred, k)
        print(f"AP {k=}:         ", AP)
    return float(AP)


In [12]:
# не изменять
ys_true = torch.tensor([1, 0, 1, 1, 0, 1, 0, 0])
ys_pred = torch.tensor([0.9, 0.8, 0.7, 0.6, 0.4, 0.3, 0.2, 0.1])

res = average_precision(ys_true, ys_pred)
print(res)  # 0.7708333333333333

------------------------
Recall k=1:      0.25
Recall k-1=0:    0.0
precission k=1:  1.0
AP k=1:          0.25
------------------------
Recall k=2:      0.25
Recall k-1=1:    0.25
precission k=2:  0.5
AP k=2:          0.25
------------------------
Recall k=3:      0.5
Recall k-1=2:    0.25
precission k=3:  0.6666666865348816
AP k=3:          0.4166666716337204
------------------------
Recall k=4:      0.75
Recall k-1=3:    0.5
precission k=4:  0.75
AP k=4:          0.6041666716337204
------------------------
Recall k=5:      0.75
Recall k-1=4:    0.75
precission k=5:  0.6000000238418579
AP k=5:          0.6041666716337204
------------------------
Recall k=6:      1.0
Recall k-1=5:    0.75
precission k=6:  0.6666666865348816
AP k=6:          0.7708333432674408
------------------------
Recall k=7:      1.0
Recall k-1=6:    1.0
precission k=7:  0.5714285969734192
AP k=7:          0.7708333432674408
------------------------
Recall k=8:      1.0
Recall k-1=7:    1.0
precission k=8:  0.5
AP 

## reciprocal_rank

```reciprocal_rank``` — функция для расчёта MRR (без усреднения). В ```ys_true``` могут содержаться только нули и максимум одна единица. 

In [123]:
def reciprocal_rank(ys_true: Tensor, ys_pred: Tensor) -> float:
    sort_idxs = torch.argsort(ys_pred, descending=True)
    pred_sorted = ys_true[sort_idxs]
    rank = torch.argmax(pred_sorted)
    return float(1 / (rank+1))

In [124]:
# не изменять
ys_true = torch.tensor([0, 0, 0, 1, 0, 0])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = reciprocal_rank(ys_true, ys_pred)
print(res)  # 0.25

0.25


## p_found

```p_found``` — функция расчёта P-found от Яндекса, принимающая на вход дополнительный параметр ```p_break``` — вероятность прекращения просмотра списка документов в выдаче. Базовая вероятность просмотреть первый документ в выдаче ($pLook[0]$) равняется единице. ```ys_true``` нормированы от 0 до 1 (вероятность удовлетворения запроса пользователя).

In [18]:
def p_found(ys_true: Tensor, ys_pred: Tensor, p_break: float = 0.15 ) -> float:
    pred_sorted, sorted_idxs = torch.sort(ys_pred, descending=False)
    true_sorted = ys_true[sorted_idxs]
    
    p_look = [1]
    for i in range(1, len(ys_true)):
        p_look += [
            p_look[i-1] * (1-true_sorted[i-1]) * (1-p_break)
        ]
    
    p_found = 0.0
    
    for i in range(len(ys_true)):
        p_found += p_look[i] * true_sorted[i]
        print(p_found)
    
    return float(p_found)

In [19]:
# не изменять
ys_true = torch.tensor([0, 0, 0, 1, 0, 1])
ys_pred = torch.tensor([0.91, 0.72, 0.12, 0.24, 0.15, 0.6])

res = p_found(ys_true, ys_pred, 0.12)
print(res)  # 0.7744

tensor(0.)
tensor(0.)
tensor(0.7744)
tensor(0.7744)
tensor(0.7744)
tensor(0.7744)
0.774399995803833
